In [1]:
import os
import pathlib
import math
import itertools
import pandas as pd
import numpy as np
from negmas import *
from negmas.apps.scml import *
from negmas.apps.scml import Input, Output, Job, FactoryStatusUpdate
from negmas.situated import Contract

In [2]:
# setup disply parameters
%pylab inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%config InlineBackend.figure_format = 'retina'
VARY_SMALL_SIZE = 24
SMALL_SIZE = 36
MEDIUM_SIZE = 48
BIGGER_SIZE = 64

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=VARY_SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
pylab.rcParams['figure.figsize'] = (24, 6)
plt.rc('lines', linewidth=3)

Populating the interactive namespace from numpy and matplotlib


In [3]:
n_steps = 10
horizon = 4
world = SCMLWorld.tiny(n_intermediate_levels=0, log_file_name='', n_miners=2, n_consumers=2, n_factories_per_level=2, n_steps=n_steps, default_signing_delay=0
                       , allow_immediate_negotiations=False, consumer_kwargs={'immediate_cfp_update': True
        , 'consumption_horizon': horizon}, negotiation_speed_multiplier=None, neg_n_steps=5)
world.run()
data=None
if len(world.saved_contracts) > 0:
    data = pd.DataFrame(data=world.saved_contracts).loc[:, ["buyer", "seller", "product", "quantity", "delivery_time"
        , "unit_price", "penalty", "signing_delay", "concluded_at", "signed_at", "issues", "cfp"]]
data

,buyer,seller,product,quantity,delivery_time,unit_price,penalty,signing_delay,concluded_at,signed_at,issues,cfp
0,c_1,f1_0,p1,1,2,4.0,NaN,0,1,1,"[time: [2], quantity: [1], unit_price: (0.0, 4...","c_1: buy p1 (t: 2, u: (0, 4.0), q: (1, 1))"
1,c_1,f1_1,p1,1,2,4.0,NaN,0,1,1,"[time: [2], quantity: [1], unit_price: (0.0, 4...","c_1: buy p1 (t: 2, u: (0, 4.0), q: (1, 1))"
2,c_1,f1_0,p1,1,3,0.0,NaN,0,1,1,"[time: [3], quantity: [1], unit_price: (0.0, 4...","c_1: buy p1 (t: 3, u: (0, 4.0), q: (1, 1))"
3,c_1,f1_1,p1,1,3,0.0,NaN,0,1,1,"[time: [3], quantity: [1], unit_price: (0.0, 4...","c_1: buy p1 (t: 3, u: (0, 4.0), q: (1, 1))"
4,c_0,f1_0,p1,1,2,0.0,NaN,0,1,1,"[time: [2], quantity: [1], unit_price: (0.0, 4...","c_0: buy p1 (t: 2, u: (0, 4.0), q: (1, 1))"
5,c_0,f1_1,p1,1,2,0.0,NaN,0,1,1,"[time: [2], quantity: [1], unit_price: (0.0, 4...","c_0: buy p1 (t: 2, u: (0, 4.0), q: (1, 1))"
6,c_0,f1_0,p1,1,3,0.0,NaN,0,1,1,"[time: [3], quantity: [1], unit_price: (0.0, 4...","c_0: buy p1 (t: 3, u: (0, 4.0), q: (1, 1))"
7,c_0,f1_1,p1,1,3,0.0,NaN,0,1,1,"[time: [3], quantity: [1], unit_price: (0.0, 4...","c_0: buy p1 (t: 3, u: (0, 4.0), q: (1, 1))"
8,c_1,f1_0,p1,1,2,0.0,NaN,0,1,1,"[time: [2], quantity: [1], unit_price: (0.0, 4...","c_1: buy p1 (t: 2, u: (0, 4.0), q: (1, 1))"
9,c_1,f1_0,p1,1,3,0.0,NaN,0,1,1,"[time: [3], quantity: [1], unit_price: (0.0, 4...","c_1: buy p1 (t: 3, u: (0, 4.0), q: (1, 1))"


In [4]:
pd.DataFrame(data=world.stats)

,n_cfps_on_board_before,n_registered_negotiations_before,n_contracts_executed,n_contracts_cancelled,n_breaches,n_contracts_signed,n_contracts_concluded,n_negotiations,n_registered_negotiations_after,n_cfps,...,storage_m_1,balance_c_0,storage_c_0,balance_c_1,storage_c_1,balance_f1_0,storage_f1_0,balance_f1_1,storage_f1_1,market_size
0,0,0,0,0,0,0,0,16,16,6,...,0,0.0,0,0.0,0,1000.0,0,1000.0,0,2000.0
1,6,16,0,0,0,16,16,0,0,4,...,0,0.0,0,0.0,0,1000.0,1,1000.0,1,2000.0
2,4,0,2,0,6,0,0,0,0,2,...,0,0.0,0,-8.0,0,998.0,0,998.0,0,1988.0
3,4,0,0,0,8,0,0,0,0,2,...,0,0.0,1,-8.0,1,990.0,0,990.0,0,1972.0
4,6,0,0,0,0,0,0,0,0,2,...,0,0.0,1,-8.0,1,990.0,0,990.0,0,1972.0
5,8,0,0,0,0,0,0,0,0,2,...,0,0.0,1,-8.0,1,990.0,0,990.0,0,1972.0
6,8,0,0,0,0,0,0,0,0,2,...,0,0.0,1,-8.0,1,990.0,0,990.0,0,1972.0
7,8,0,0,0,0,0,0,0,0,0,...,0,0.0,1,-8.0,1,990.0,0,990.0,0,1972.0
8,6,0,0,0,0,0,0,0,0,0,...,0,0.0,1,-8.0,1,990.0,0,990.0,0,1972.0
9,4,0,0,0,0,0,0,0,0,0,...,0,0.0,1,-8.0,1,990.0,0,990.0,0,1972.0
